# Preparing an AF state in the Ising model

This notebook illustrates how to use Pulser to build a sequence for preparing an AF state in the Ising model. It is based on [10.1103/PhysRevX.8.021070](10.1103/PhysRevX.8.021070).

We begin by importing some basic modules:

In [ ]:
import numpy as np
import itertools
import matplotlib.pyplot as plt
import qutip

from pulser import Pulse, Sequence, Register
from pulser.waveforms import ConstantWaveform, RampWaveform
from pulser.devices import Chadoq2, MockDevice
from pulser.simulation import Simulation

# 1. 2D Square Array

## Waveforms 

We are realizing the following program

<img src="files/AF_Ising_program.png" alt="AF Pulse Sequence" style="width: 320px;"/>

This pulse is defined by the following parameters:

In [ ]:
# Parameters in MHz and ns
U = 1.5 * 2*np.pi 

delta_0 = -20 * U
delta_f = 14 * U
Omega_max = 12 * U 

t_rise = 300
t_fall = 400
t_sweep = 1000 - (t_rise + t_fall)

We define our register with an interatomic distance equal to the Rydberg radius. This in turn depends on the maximum Rabi frequency in our pulse sequence.

In [ ]:
R_blockade = (5.008e6/Omega_max)**(1/6)

N = 3
reg = Register.rectangle(N, N, R_blockade, prefix='q')
print(f'Blockade Radius is: {R_blockade}µm.')
reg.draw()

## Creating my sequence

We compose our pulse with the following objects from Pulser:

In [ ]:
rise = Pulse.ConstantDetuning(RampWaveform(t_rise, 0., Omega_max), delta_0, 0.)
sweep = Pulse.ConstantAmplitude(Omega_max, RampWaveform(t_sweep, delta_0, delta_f), 0.)
fall = Pulse.ConstantDetuning(RampWaveform(t_fall, Omega_max, 0.), delta_f, 0.)

In [ ]:
seq = Sequence(reg, MockDevice)
seq.declare_channel('ising', 'rydberg_global')

seq.add(rise, 'ising')
seq.add(sweep, 'ising')
seq.add(fall, 'ising')

#print(seq)
seq.draw()

## Phase Diagram

The pulse sequence travels though the following path in the phase diagram of the system (the shaded area represents the antiferromagnetic phase):

In [ ]:
#phase = {'omega':[], 'delta':[], 'time': range(max(seq._last(ch).tf for ch in seq.declared_channels))}
delta = []
omega = []
for x in seq._schedule['ising']:
    if isinstance(x.type,Pulse):
        omega += list(x.type.amplitude.samples/U)
        delta += list(x.type.detuning.samples/U)
        
fig, ax = plt.subplots()
ax.grid(True, which='both')

ax.set_ylabel(r"$\frac{\delta(t)}{\hbar\,U}$", fontsize=12)
ax.set_xlabel(r"$\frac{\Omega(t)}{\hbar\,U}$", fontsize=12)
ax.set_xlim(0,3)
#ax.set_ylim(-300,300)
ax.axhline(y=0, color='k')
ax.axvline(x=0, color='k')

y = np.arange(0.0, 6, 0.01)
x = 1.522*(1-0.25*(y-2)**2)
ax.fill_between(x, y, alpha=0.4)

ax.plot(np.array(omega)/(2*np.pi),np.array(delta)/(2*np.pi),'red',lw=2)
plt.show()


# Simulation

We now run a simulation of the sequence:

In [ ]:
simul = Simulation(seq)

The observables to measure will be the occupation operator $|r\rangle \langle r|_i$ on each site $i$ of the register, where the Rydberg state $|r\rangle$ represents the excited state.

In [ ]:
simul._size

In [ ]:
up = qutip.basis(2,0)
def occupation(N,j):
    prod = [qutip.qeye(2) for _ in range(N*N)]
    prod[j] = up*up.dag()
    return qutip.tensor(prod)

In [ ]:
occup_list = [occupation(N,j) for j in range(N*N)]

We now run the simulation and plot the evolution of the expectation values, as well as a representation of the final values after the pulse is applied:

In [ ]:
simul.run(obs_list=occup_list, progress_bar=True)

for expv in simul.output.expect:
    plt.plot(expv)
    
res=np.zeros((N,N))
for i,ev in enumerate(simul.output.expect):
    res[i//N,i%N] = ev[-1]
plt.matshow(res, cmap='hot')
print(res)

## Spin-Spin Correlation Function

In [ ]:
simul.run(progress_bar=True)

In [ ]:
def get_corr_pairs(k, l, reg,R_blockade):
    corr_pairs = []
    for i, qi in enumerate(reg.qubits):
        for j, qj in enumerate(reg.qubits):
            if all((reg.qubits[qi]-reg.qubits[qj])/R_blockade == [k,l]):
                corr_pairs.append([i,j])
    return corr_pairs

In [ ]:
def get_corr_function(N,k,l,reg,R_blockade, state):
    covariance = 0
    corr_pairs = get_corr_pairs(k,l, reg,R_blockade)
    occup_list = [occupation(N,j) for j in range(N*N)]
    for qi, qj in corr_pairs:
        covariance += qutip.expect(occup_list[qi]*occup_list[qj], state)
        covariance -= qutip.expect(occup_list[qi], state)*qutip.expect(occup_list[qj], state)
    return covariance/len(corr_pairs)
        
def get_full_corr_function(N,reg,state):
    correlation_function = {}
    for k,l in itertools.product(range(-N+1,N), repeat=2):
        correlation_function[(k,l)] = get_corr_function(N, k, l, reg,R_blockade,state)
    return correlation_function

In [ ]:
final = simul.output.states[-1]
correlation_function = get_full_corr_function(N, reg, final)

In [ ]:
expected_corr_function = {}
xi = 1.5
for k, l in itertools.product(range(-N+1,N), repeat=2):
    kk = np.abs(k)
    ll = np.abs(l)
    expected_corr_function[(k,l)] = (-1)**(kk + ll) * np.exp(-(kk + ll)/xi)

In [ ]:
A = 4*np.reshape(list(correlation_function.values()), (2*N-1,2*N-1))
B = 0.06*np.reshape(list(expected_corr_function.values()), (2*N-1,2*N-1))

fig, axes = plt.subplots(nrows=2, ncols=1)
im_A = axes[0].imshow(A, vmin=-0.25, vmax=0.25, cmap='coolwarm')
axes[0].set_title(r'$g^{(2)}(k,l)$ after simulation')

im_B = axes[1].imshow(B, vmin=-0.25, vmax=0.25, cmap='coolwarm')
#axes[1].set_title(r'Exponential $g^{(2)}(k,l)$ expected')

fig.colorbar(im_A, ax=axes.ravel().tolist())
plt.show()


## Néel Structure Factor

In [ ]:
def get_neel_structure_factor(N,reg,R_blockade,state):
    st_fac = 0
    for k,l in itertools.product(range(-N+1,N), repeat=2):
        kk = np.abs(k)
        ll = np.abs(l)
        if not (k == 0 and l == 0):
            st_fac += 4 * (-1)**(kk + ll) * get_corr_function(N,k,l,reg,R_blockade,state)
    return st_fac 

In [ ]:
def calculate_neel(det, omax=None):
    # Parameters in MHz and ns 
    U = 1.5 * 2*np.pi
    if omax:
        Omega_max=omax*U
    else:
        Omega_max = 9*U
    delta_0 = -10 * U
    delta_f = det * U

    R_blockade = (5.008e6/Omega_max)**(1/6)

    N = 3
    reg = Register.rectangle(N, N, R_blockade, prefix='q')
    #print(f'Blockade Radius is: {R_blockade}µm.')
    #reg.draw()


    t_rise = 300
    t_fall = 400
    t_sweep = 1000 - (t_rise + t_fall)


    rise = Pulse.ConstantDetuning(RampWaveform(t_rise, 0., Omega_max), delta_0, 0.)
    sweep = Pulse.ConstantAmplitude(Omega_max, RampWaveform(t_sweep, delta_0, delta_f), 0.)
    fall = Pulse.ConstantDetuning(RampWaveform(t_fall, Omega_max, 0.), delta_f, 0.)

    seq = Sequence(reg, MockDevice)
    seq.declare_channel('ising', 'rydberg_global')

    seq.add(rise, 'ising')
    seq.add(sweep, 'ising')
    seq.add(fall, 'ising')

    #print(seq)
    #seq.draw()

    simul = Simulation(seq)
    simul.run()
    
    final = simul.output.states[-1]
    return get_neel_structure_factor(N,reg,R_blockade,final)

                                                                                                                                                                      

In [ ]:
detunings = np.linspace(-3,7,10)
omegas = np.linspace(1,6,4)
for omax in omegas:
    results=[]
    for det in detunings:
        results.append(calculate_neel(det, omax=omax))
    plt.xlabel(r'$\hbar\delta_{final}/U$')
    plt.title(r'Néel Structure Factor $S_{Neel}$')
    plt.plot(detunings, results, 'o', ls='solid', label=rf'$\Omega$={omax}')
    plt.legend()

# 2. 1D open Chain

One can also try other geometries. In a 1D open boundary chain, for example:

In [ ]:
# Parameters in MHz and ns
U = 1.5 * 2*np.pi 
delta_0 = -10 * U
delta_f = 7 * U
Omega_max = 10 * U
t_rise = 300
t_fall = 400
t_sweep = 1000 - (t_rise+t_fall)

# Register
R_blockade = (5.008e6/Omega_max)**(1/6)
n_side = 5
coords = R_blockade * np.array([(x,y) for x in range(n_side) for y in range(n_side) 
                    if (x in {0,n_side-1} or y in {0,n_side-1})], dtype=np.float64)
reg = Register.from_coordinates(coords, prefix='atom')
reg = Register.rectangle(1,11,R_blockade, prefix='q')

reg.draw()


rise = Pulse.ConstantDetuning(RampWaveform(t_rise, 0., Omega_max), delta_0, 0.)
sweep = Pulse.ConstantAmplitude(Omega_max, RampWaveform(t_sweep, delta_0, delta_f), 0.)
fall = Pulse.ConstantDetuning(RampWaveform(t_fall, Omega_max, 0.), delta_f, 0.)

seq = Sequence(reg, MockDevice)
seq.declare_channel('ising', 'rydberg_global')

seq.add(rise, 'ising')
seq.add(sweep, 'ising')
seq.add(fall, 'ising')

#print(seq)
seq.draw()

The phase diagram is plotted below:

In [ ]:
#phase = {'omega':[], 'delta':[], 'time': range(max(seq._last(ch).tf for ch in seq.declared_channels))}
delta = []
omega = []
for x in seq._schedule['ising']:
    if isinstance(x.type,Pulse):
        omega += list(x.type.amplitude.samples/U)
        delta += list(x.type.detuning.samples/U)
        
fig, ax = plt.subplots()
ax.grid(True, which='both')

ax.set_ylabel(r"$\frac{\delta(t)}{\hbar\,U}$")
ax.set_xlabel(r"$\frac{\Omega(t)}{\hbar\,U}$")
ax.set_xlim(0,2)
ax.set_ylim(-3,3)
ax.axhline(y=0, color='k')
ax.axvline(x=0, color='k')

y = np.arange(0.0, 6, 0.01)
x = 0.5*(1-(y-1)**2)
ax.fill_between(x, y, color='green',alpha=0.4)

ax.plot(np.array(omega)/(2*np.pi),np.array(delta)/(2*np.pi),'red',lw=2)

In [ ]:
simul = Simulation(seq)

up = qutip.basis(2,0)
def occupation(j):
    prod = [qutip.qeye(2) for _ in range(simul._size)]
    prod[j] = up*up.dag()
    return qutip.tensor(prod)
    
occup_list = [occupation(j) for j in range(simul._size)]

simul.run(obs_list=occup_list, progress_bar=True)

The results are as follows:

In [ ]:
for expv in simul.output.expect:
    plt.plot(expv)
plt.show()
L = len(reg.qubits)
res =np.zeros((1,L))
for i,ev in enumerate(simul.output.expect):
    res[0,i] = ev[-1]
plt.matshow(res, cmap = 'hot')
print(res)